<a href="https://colab.research.google.com/github/JacobDowns/glamacles_test/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
a = np.linspace(0.,1.,10)